In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm


### 1. Reading raw data 

In [3]:
#gdis data 
gdis = pd.read_csv('../../data/pend-gdis-1960-2018-disasterlocations.csv')
#get emdat dataset
emdat = pd.read_csv('../../data/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])
gdis.head(2)

<ipython-input-3-1a121b5983b0>:4: DtypeWarning: Columns (8,16,17,18,19,24,25,26,27,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  emdat = pd.read_csv('../../data/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])


,id,country,iso3,gwno,year,geo_id,geolocation,level,adm1,adm2,adm3,location,historical,hist_country,disastertype,disasterno,latitude,longitude
0,109,Albania,ALB,339.0,2009,346,Ana E Malit,3,Shkoder,Shkodres,Ana E Malit,Ana E Malit,0,NaN,flood,2009-0631,42.020948,19.418317
1,109,Albania,ALB,339.0,2009,351,Bushat,3,Shkoder,Shkodres,Bushat,Bushat,0,NaN,flood,2009-0631,41.959294,19.514309


In [4]:
# This is where we obtain the statistical features
emdat.head(2)

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN


In [5]:
#select certain columns from emdat and join with gdis 
# we grab the disaster number and convert it into string format and grab everything except for the last 4 ( so only the dates)
emdat['disasterno'] = emdat['Dis No'].str[:-4] #format disasterno to merge  

#These are the columns that we want to keep  , the features below are left out!
cols = ['disasterno', 'Year', 'Event Name', 
#         'Disaster Type', 'Disaster Subtype', 
#         'Region', 'Continent', #'Location',
        'Start Year', 'Start Month', 'Start Day', 
        'End Year', 'End Month','End Day',  
        "Total Damages, Adjusted ('000 US$)"] 

emdat = emdat[cols]


In [6]:
#join emdat and gdis into one dataframe
gdis = pd.merge(emdat, gdis, on = 'disasterno', how='right')

In [7]:
gdis.head(3)

,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,level,adm1,adm2,adm3,location,historical,hist_country,disastertype,latitude,longitude
0,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,3,Shkoder,Shkodres,Ana E Malit,Ana E Malit,0,NaN,flood,42.020948,19.418317
1,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,3,Shkoder,Shkodres,Bushat,Bushat,0,NaN,flood,41.959294,19.514309
2,2001-0146,2001.0,NaN,2001.0,4.0,2.0,2001.0,4.0,9.0,NaN,...,3,Cunene,Cuanhama,Onjiva,Onjiva,0,NaN,flood,-17.093484,15.665758


In [8]:
#print 
gdis = gdis.drop_duplicates(subset=['id'])
print('shape', gdis.shape)
gdis.head(2)

shape (9924, 27)


,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,level,adm1,adm2,adm3,location,historical,hist_country,disastertype,latitude,longitude
0,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,3,Shkoder,Shkodres,Ana E Malit,Ana E Malit,0,NaN,flood,42.020948,19.418317
2,2001-0146,2001.0,NaN,2001.0,4.0,2.0,2001.0,4.0,9.0,NaN,...,3,Cunene,Cuanhama,Onjiva,Onjiva,0,NaN,flood,-17.093484,15.665758


In [9]:
# old grid_id 
#latitude range from -90 to 90, longitude range from -180 to 180 

# #convert lat and long into 1 degree grid: correspond to 100*100km 
# gdis['lat_grid_old'] = np.digitize(np.array(gdis['latitude']),np.arange(-90,90,1))
# gdis['lon_grid_old'] = np.digitize(np.array(gdis['longitude']),np.arange(-180,180,1)) 
# #compute the grid pair id 
# gdis['grid_id_old'] = list(zip(gdis['lat_grid_old'],gdis['lon_grid_old']))
# print('total number of grid pairs', len(gdis.grid_id.value_counts()))
# gdis[['grid_id_old','grid_id']].to_csv('grid_id_conversion.csv')

In [10]:
#new grid_id: round to integers 
gdis['lat_grid'] = gdis['latitude'].round().astype(int)
gdis['lon_grid'] = gdis['longitude'].round().astype(int)
gdis['grid_id'] = list(zip(gdis['lat_grid'],gdis['lon_grid'])) 
print('total number of grid pairs', len(gdis.grid_id.value_counts()))

# check if they lie in range
print('lon range', gdis['lon_grid'].min(), gdis['lon_grid'].max()) 
print('lat range', gdis['lat_grid'].min(), gdis['lat_grid'].max()) 

total number of grid pairs 2827
lon range -178 180
lat range -54 68


In [11]:
gdis.head(2)

,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,adm3,location,historical,hist_country,disastertype,latitude,longitude,lat_grid,lon_grid,grid_id
0,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,Ana E Malit,Ana E Malit,0,NaN,flood,42.020948,19.418317,42,19,"(42, 19)"
2,2001-0146,2001.0,NaN,2001.0,4.0,2.0,2001.0,4.0,9.0,NaN,...,Onjiva,Onjiva,0,NaN,flood,-17.093484,15.665758,-17,16,"(-17, 16)"


In [12]:
# #count number of locations in each grid_id: ideally just 1 location per grid_id, if not, I can make grid finer. 
# gdis.groupby('grid_id').agg({'location':'nunique'}).sort_values(by='location').value_counts()

In [13]:
# # the grid with most regions 
# gdis.loc[gdis['grid_id']==(113, 295)].head(5)

### 2. Construct X

In [14]:
#pivot function: change rows of info into tables 
def pivot(df_in, id_col='disastertype', id_list=['Flood']):
    #Drop and reset the index column
    df = df_in.reset_index(drop = True)

    # one set of disaster
    for id in id_list:
        #initialize columns
        df[id+'_bin'] = 0
        df[id+'_amt'] = 0
        df[id+'_ct'] = 0
        
        
        # so if the flood happens and we find it, we create these columns and get the flood costs
        df.loc[(df[id_col]==id), id+'_bin'] = 1
        df.loc[(df[id_col]==id), id+'_amt'] = df["Total Damages, Adjusted ('000 US$)"].astype(float)
        df.loc[(df[id_col]==id), id+'_ct'] = 1

    return df


In [15]:
# id_list= df_sub['Disaster Type'].unique()
id_list= gdis['disastertype'].unique().tolist()
print(id_list)
df_pivot= pivot(gdis, id_col = 'disastertype', id_list = id_list)
df_pivot


['flood', 'storm', 'earthquake', 'extreme temperature ', 'landslide', 'volcanic activity', 'drought', 'mass movement (dry)']


,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,landslide_ct,volcanic activity_bin,volcanic activity_amt,volcanic activity_ct,drought_bin,drought_amt,drought_ct,mass movement (dry)_bin,mass movement (dry)_amt,mass movement (dry)_ct
0,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,0,0,0.0,0,0,0.0,0,0,0.0,0
1,2001-0146,2001.0,NaN,2001.0,4.0,2.0,2001.0,4.0,9.0,NaN,...,0,0,0.0,0,0,0.0,0,0,0.0,0
2,2009-0092,2009.0,NaN,2009.0,3.0,1.0,2009.0,4.0,16.0,NaN,...,0,0,0.0,0,0,0.0,0,0,0.0,0
3,2010-0105,2010.0,NaN,2010.0,3.0,1.0,2010.0,3.0,17.0,NaN,...,0,0,0.0,0,0,0.0,0,0,0.0,0
4,1995-0082,1995.0,NaN,1995.0,5.0,15.0,1995.0,5.0,15.0,1422569.0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9919,1960-0011,1960.0,Manam,1960.0,3.0,17.0,1960.0,3.0,17.0,NaN,...,0,1,NaN,1,0,0.0,0,0,0.0,0
9920,2009-9633,2009.0,NaN,2009.0,NaN,NaN,2010.0,NaN,NaN,NaN,...,0,0,0.0,0,1,NaN,1,0,0.0,0
9921,1990-9289,1990.0,NaN,1990.0,NaN,NaN,1990.0,NaN,NaN,NaN,...,0,0,0.0,0,1,NaN,1,0,0.0,0
9922,1969-9069,1969.0,NaN,1969.0,NaN,NaN,1971.0,NaN,NaN,73867.0,...,0,0,0.0,0,1,73867.0,1,0,0.0,0


In [16]:
df_pivot.columns

Index(['disasterno', 'Year', 'Event Name', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Damages, Adjusted ('000 US$)', 'id', 'country', 'iso3', 'gwno',
       'year', 'geo_id', 'geolocation', 'level', 'adm1', 'adm2', 'adm3',
       'location', 'historical', 'hist_country', 'disastertype', 'latitude',
       'longitude', 'lat_grid', 'lon_grid', 'grid_id', 'flood_bin',
       'flood_amt', 'flood_ct', 'storm_bin', 'storm_amt', 'storm_ct',
       'earthquake_bin', 'earthquake_amt', 'earthquake_ct',
       'extreme temperature _bin', 'extreme temperature _amt',
       'extreme temperature _ct', 'landslide_bin', 'landslide_amt',
       'landslide_ct', 'volcanic activity_bin', 'volcanic activity_amt',
       'volcanic activity_ct', 'drought_bin', 'drought_amt', 'drought_ct',
       'mass movement (dry)_bin', 'mass movement (dry)_amt',
       'mass movement (dry)_ct'],
      dtype='object')

In [17]:
#aggregate columns by year
# We need to include the longitude and lattitude here 
def aggregate_yrly(df):
    #aggregate count
    col_ct = [col for col in df.columns if '_ct' in col]
    df_ct = df.groupby(['grid_id','year'])[col_ct].agg('sum')
    
    #aggregate amount 
    col_amt = [col for col in df.columns if '_amt' in col]
    df_amt = df.groupby(['grid_id','year'])[col_amt].agg('sum')
    
    #aggregate binary
    col_bin = [col for col in df.columns if '_bin' in col]
    df_bin= df.groupby(['grid_id','year'])[col_bin].agg('max')

    #join
    df1= pd.concat([df_amt, df_ct], axis=1)
    df_out = pd.concat([df1, df_bin], axis=1)
    return df_out.reset_index()



df_yrly = aggregate_yrly(df_pivot)


In [18]:
df_yrly.columns

Index(['grid_id', 'year', 'flood_amt', 'storm_amt', 'earthquake_amt',
       'extreme temperature _amt', 'landslide_amt', 'volcanic activity_amt',
       'drought_amt', 'mass movement (dry)_amt', 'flood_ct', 'storm_ct',
       'earthquake_ct', 'extreme temperature _ct', 'landslide_ct',
       'volcanic activity_ct', 'drought_ct', 'mass movement (dry)_ct',
       'flood_bin', 'storm_bin', 'earthquake_bin', 'extreme temperature _bin',
       'landslide_bin', 'volcanic activity_bin', 'drought_bin',
       'mass movement (dry)_bin'],
      dtype='object')

In [19]:
print(len(df_yrly))

8691


In [20]:
# We want to remove years that are before 1979 as we don't have data for those in google earths
df_yrly = df_yrly.loc[df_yrly['year']>1979].copy().reset_index(drop=True)
print(len(df_yrly))

7977


#### Google earth functions

In [39]:
# Authenticating google earths and importing it
import ee

import warnings
warnings.filterwarnings("ignore")

In [22]:
# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/1AWtgzh5iKXFgD4DRocd6BJY667IX3Kt6p_xFxM997ejGt8eHTmP6icXHIGM

Successfully saved authorization token.


In [23]:
# Initialize the library.
ee.Initialize()

In [24]:
# importing the right dataset
era5_monthly = ee.ImageCollection("ECMWF/ERA5/MONTHLY")

In [25]:
def ee_array_to_df(arr, list_of_bands):
    """
    This is a helper function that transforms client-side ee.Image.getRegion array to pandas.DataFrame. 
    Used in the main function later
    Input:
        Takes in the array and the list of bands we want to extract
        Takes in the array we are dealing with 
    Returns:
        Dataframe we want in time-series format
    """
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]
    

    return df 

In [26]:
def features_df(i_date,f_date,imagecollec,long,lat,scale=100,list_of_bands = ['mean_2m_air_temperature','minimum_2m_air_temperature','maximum_2m_air_temperature','dewpoint_2m_temperature','total_precipitation','surface_pressure','mean_sea_level_pressure','u_component_of_wind_10m','v_component_of_wind_10m']):
    """
    Function that produces the time-series in dataframe of the different features we need from the dataset 
    
    Input:
        - List of features
        - Start and end date we are interested in with the format '2017-01-01'
        - the image collection in question
        - Longitude - Float
        - Lattitude - Float
        - Scale - Float
    Returns:
        Dataframe we want in time-series format
    """
    # Filter by the dates
    imagecollec_sel_dates = imagecollec.filterDate(i_date, f_date)
    
    # Creation of the geometry object

    point = ee.Geometry.Point(long, lat)
    
    # To download the time-series data
    time_series_list = imagecollec_sel_dates.getRegion(point, scale).getInfo()
    
    # Convert the time_series list to dataframe and return it
    output_df = ee_array_to_df(time_series_list,list_of_bands)
        
    return output_df
    

In [27]:
def reformat_features(era_features): 
    era_features['month'] = era_features['datetime'].dt.month.astype(str)
    era_features = era_features.drop(['time','datetime'], axis=1).set_index('month')
    row_data = era_features.stack().values
    row_cols = era_features.stack().index.map('_'.join).tolist()
    era_features_row = pd.DataFrame(columns=row_cols, data=row_data.reshape(1,-1))
    return era_features_row 

year = 2005
i_date = str(year)+'-01-01'
f_date = str(year+1)+'-01-01' 
lat, long = (1,1)
imagecollec = era5_monthly
#call features_df to get era features for a range of dates at a particular lat and long 
era_features_one = features_df(i_date,f_date,imagecollec,long,lat) 
#convert this into a row of features, with column name being the month_feature_name 
era_features_row = reformat_features(era_features_one) 
era_features_row

,1_mean_2m_air_temperature,1_minimum_2m_air_temperature,1_maximum_2m_air_temperature,1_dewpoint_2m_temperature,1_total_precipitation,1_surface_pressure,1_mean_sea_level_pressure,1_u_component_of_wind_10m,1_v_component_of_wind_10m,2_mean_2m_air_temperature,...,11_v_component_of_wind_10m,12_mean_2m_air_temperature,12_minimum_2m_air_temperature,12_maximum_2m_air_temperature,12_dewpoint_2m_temperature,12_total_precipitation,12_surface_pressure,12_mean_sea_level_pressure,12_u_component_of_wind_10m,12_v_component_of_wind_10m
0,300.250275,298.078094,301.828064,296.496399,0.075643,101028.09375,101028.21875,0.688926,1.466673,300.750916,...,4.113983,299.566833,298.023041,301.030975,296.309631,0.128574,100978.570312,100978.6875,0.05138,3.885582


In [30]:
df_yrly.year.max()

2018

In [51]:
#interpolate all years from 1979 (when ERA5 data is available)
#filter to those grid_ids with previous frequent flooding history 
agg = df_yrly.groupby('grid_id').agg({'flood_bin':'sum'})
grid_id_ls = agg.loc[agg['flood_bin']>=2].index.tolist()
print('no of grid_ids selected', len(grid_id_ls))
year_id = np.arange(1979, 2020, 1) 
print(len(grid_id_ls))
idd = pd.MultiIndex.from_product([grid_id_ls, year_id],
                           names=['grid_id', 'year']).to_frame().reset_index(drop=True)
idd

no of grid_ids selected 791
791


,grid_id,year
0,"(-46, 168)",1979
1,"(-46, 168)",1980
2,"(-46, 168)",1981
3,"(-46, 168)",1982
4,"(-46, 168)",1983
...,...,...
32426,"(66, 130)",2015
32427,"(66, 130)",2016
32428,"(66, 130)",2017
32429,"(66, 130)",2018


In [53]:
#I already ran this cell, it takes 1h 
i=0 
era_features =pd.DataFrame()
for index, row in tqdm(idd.iterrows(), total=idd.shape[0]):
    year = row['year']
    i_date = str(year)+'-01-01'
    f_date = str(year+1)+'-01-01' 
    lat, long = row['grid_id']
    #call features_df to get era features for a range of dates at a particular lat and long 
    era_features_one = features_df(i_date,f_date,imagecollec, long+0.5, lat+0.5) 
    #convert this into a row of features, with column name being the month_feature_name 
    era_features_row = reformat_features(era_features_one) 
    #append year, lat and long info to get grid_id 
    era_features_row[['year', 'lat', 'long']] = year, lat, long

    era_features = era_features.append(era_features_row, ignore_index=True)
    i +=1 
    #save output every 1000 iterations 
    if i in range(1,len(idd),5000):
        era_features.to_csv('era_features.csv')

100%|██████████| 32431/32431 [3:53:12<00:00,  2.32it/s]   


In [54]:
#save extracted features into csv 
era_features
era_features.to_csv('era_features.csv')

In [55]:
# read era_features (ran from above cell)
era_features = pd.read_csv('era_features.csv', index_col=0)
era_features.head()

,1_mean_2m_air_temperature,1_minimum_2m_air_temperature,1_maximum_2m_air_temperature,1_dewpoint_2m_temperature,1_total_precipitation,1_surface_pressure,1_mean_sea_level_pressure,1_u_component_of_wind_10m,1_v_component_of_wind_10m,2_mean_2m_air_temperature,...,12_maximum_2m_air_temperature,12_dewpoint_2m_temperature,12_total_precipitation,12_surface_pressure,12_mean_sea_level_pressure,12_u_component_of_wind_10m,12_v_component_of_wind_10m,year,lat,long
0,283.617432,274.514099,293.051849,279.182190,0.215081,92250.945312,101025.101562,2.339942,-1.741069,283.682892,...,293.280151,278.507690,0.214973,91823.835938,100589.523438,1.331599,-1.586958,1979,-46,168
1,284.689545,273.895508,295.293182,280.788971,0.264142,92111.539062,100855.046875,1.082877,-0.996928,282.993622,...,293.592224,280.121765,0.088443,92457.976562,101241.312500,0.833229,-0.764634,1980,-46,168
2,284.936646,274.029205,293.806641,280.351562,0.065239,92711.820312,101490.492188,1.865336,-1.452276,284.757507,...,293.230774,281.170258,0.208652,92287.460938,101046.523438,1.027116,-0.874772,1981,-46,168
3,283.649567,274.507050,293.252380,279.620819,0.219091,92001.867188,100763.882812,2.079320,-1.756939,284.861938,...,292.498871,277.305939,0.160083,91935.570312,100772.304688,1.964107,-1.288524,1982,-46,168
4,282.124695,272.930054,293.302368,278.383118,0.222093,91664.773438,100442.992188,1.911922,-2.058668,283.099060,...,294.530212,277.901886,0.195903,92046.609375,100889.664062,1.336580,-0.468514,1983,-46,168


In [23]:
#testing: 




In [24]:
#statistical features only 


### Construct Y Master
look up table for all the previous flood events

In [35]:
bin_col =  [col for col in df_yrly.columns if '_bin' in col]
df_yrly_bin = df_yrly[['grid_id','year'] + bin_col]


In [36]:
# #get a list of disasters and flood_id 
# all_dis = gdis[['grid_id','Year','disastertype']]

# #
# flood = all_dis.loc[all_dis['disastertype']=='flood']
# flood

In [37]:
# get a list of grid_ids 
grid_id = gdis['grid_id'].unique()
# get a list of year information 
print(gdis.Year.min(), gdis.Year.max())
year_id = np.arange(1960, 2019, 1) 
#create multi-index: each grid id, spanning over the years 
idd = pd.MultiIndex.from_product([grid_id, year_id],
                           names=['grid_id', 'year'])

#length should be |years| * |grid_ids| 
print(len(idd))
#get dataframe 
idd = idd.to_frame().reset_index(drop=True)


1960.0 2018.0
166793


In [38]:
#master disaster targets for all years and all ids: 
#merge with df_yrly 
y_master = pd.merge(idd, df_yrly_bin, on=['grid_id','year'], how='left').fillna(0)

#keep just the binary
# y_master.sum(axis=0)

In [39]:
#check: 
print(y_master['flood_bin'].sum()) #total number of binary flood targets 

#total number of flood incidents: 
print(gdis.loc[gdis['disastertype']=='flood'].shape[0])

#the two numbers are slightly different, but that's because some country have more than 1 flood per year 



3783.0
4274


In [40]:
# construct next n year target -> look up this table 
y_master


,grid_id,year,flood_bin,storm_bin,earthquake_bin,extreme temperature _bin,landslide_bin,volcanic activity_bin,drought_bin,mass movement (dry)_bin
0,"(42, 19)",1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(42, 19)",1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(42, 19)",1962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(42, 19)",1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"(42, 19)",1964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
166788,"(9, 30)",2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166789,"(9, 30)",2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166790,"(9, 30)",2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166791,"(9, 30)",2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Filter data to previously flooded regions 


In [41]:
#step 1: filter xy_df to those grid_ids with previous frequent flooding history 
agg = df_yrly.groupby('grid_id').agg({'flood_bin':'sum'})
grid_id_ls = agg.loc[agg['flood_bin']>=2].index.tolist()
print('no of grid_ids selected', len(grid_id_ls))


#step 2: interpolate years to record all years, fill with 0 without any flood using idd 
#create multi-index: each grid id, spanning over the years 
year_id = np.arange(1960, 2019, 1) 
idd = pd.MultiIndex.from_product([grid_id_ls, year_id],
                           names=['grid_id', 'year'])

#length should be |years| * |grid_ids| 
print('len of idd', len(idd))
#get dataframe 
idd = idd.to_frame().reset_index(drop=True)


no of grid_ids selected 791
len of idd 46669


In [42]:
df_yrly.shape


(7977, 26)

### attach NLP feature

In [86]:
import pickle 
print(pickle.format_version)

#load processed nlp features: 
#note: you can load other nlp features too, they all start with 'nlp_*' 
file = open('../../data/nlp_cls_transfer.pkl', 'rb') 
# load file
df_nlp = pickle.load(file)
# close the file
file.close()
df_nlp.shape

4.0


(2852, 37)

In [87]:
#convert old grid id to new grid id 
grid_id_map = pd.read_csv('grid_id_conversion.csv', index_col=0)
grid_id_map = grid_id_map.drop_duplicates(subset=['grid_id_old'], keep='first')
df_nlp = df_nlp.rename(columns={'grid_id':'grid_id_old'})
print(df_nlp.shape, grid_id_map.shape)
df_nlp = df_nlp.merge(grid_id_map, on='grid_id_old', how='left').drop('grid_id_old', axis=1)
print(df_nlp.shape)
#drop duplicates 
df_nlp = df_nlp.drop_duplicates(subset='grid_id', keep='first')
print(df_nlp.shape)

(2852, 37) (2852, 2)
(2852, 37)
(2382, 37)


In [74]:
#fixing formatting error: change string of tuple of grid_id to tuple 
def string_to_tuple(df, col): 
    try: 
        df[col] = df.apply(lambda row: eval(row[col]), axis=1) 
    except: 
        'error converting to tuple'
    return df 


In [91]:
#attach nlp to xy_df 
def attach_nlp(xy_df, df_nlp):
    #correct formatting for df_nlp 
    df_nlp = string_to_tuple(df_nlp, 'grid_id')
    #drop text, location and label columns 
    df_nlp = df_nlp.drop(['location','txt','label','flood_ct_x'], axis=1)
    #add prefix 
    df_nlp = df_nlp.rename(columns={c: 'nlp_' + str(c) for c in df_nlp.columns if c not in ['grid_id']})
    #merge 
    xy_df_out = pd.merge(xy_df, df_nlp , on='grid_id', how='left')
    print('shape of xy_df with nlp features', xy_df_out.shape)
    return xy_df_out 

attach_nlp(xy_df, df_nlp)

shape of xy_df with nlp features (7759, 59)


,grid_id,year,flood_amt,storm_amt,earthquake_amt,extreme temperature _amt,landslide_amt,volcanic activity_amt,drought_amt,mass movement (dry)_amt,...,nlp_22,nlp_23,nlp_24,nlp_25,nlp_26,nlp_27,nlp_28,nlp_29,nlp_30,nlp_31
0,"(-54, -72)",1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.285712,0.175260,0.066858,0.402230,0.307911,0.413373,-0.304808,0.332228,0.176272,-0.206565
1,"(-49, -73)",2017,2211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.295375,0.147407,-0.000792,0.327370,0.290429,0.431107,-0.327519,0.183837,0.048896,-0.185709
2,"(-49, -70)",1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.375935,0.161357,0.086125,0.396205,0.309829,0.434621,-0.271017,0.377438,0.124515,-0.173165
3,"(-46, -73)",2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.303552,0.173102,0.097518,0.477680,0.316651,0.445075,-0.281092,0.293919,0.158472,-0.205128
4,"(-46, -73)",2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.303552,0.173102,0.097518,0.477680,0.316651,0.445075,-0.281092,0.293919,0.158472,-0.205128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7754,"(66, 130)",1998,5818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.241954,0.260355,0.086829,0.508838,0.289577,0.384660,-0.194955,0.296642,0.106905,-0.120921
7755,"(66, 130)",2007,33655.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.241954,0.260355,0.086829,0.508838,0.289577,0.384660,-0.194955,0.296642,0.106905,-0.120921
7756,"(66, 130)",2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.241954,0.260355,0.086829,0.508838,0.289577,0.384660,-0.194955,0.296642,0.106905,-0.120921
7757,"(68, -135)",2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.296960,0.232861,0.145010,0.493256,0.297104,0.419325,-0.291666,0.335292,0.182556,-0.052409


### Attach ERA features 

In [139]:
# read era_features (ran from above cell)
df_era = pd.read_csv('era_features.csv', index_col=0)
df_era['grid_id'] = list(zip(df_era['lat'],df_era['long']))

In [140]:
#attach era features
def attach_era(xy_df, df_era):
    #add prefix 
    df_era = df_era.rename(columns={c: 'era_' + c for c in df_era.columns if c not in ['grid_id','year']})
    xy_df_out = pd.merge(xy_df, df_era, on=['grid_id','year'], how='left')
    print('shape of xy_df with era features', xy_df_out.shape)
    return xy_df_out

attach_era(xy_df, df_era)

shape of xy_df with era features (45878, 279)


,grid_id,year,flood_amt,storm_amt,earthquake_amt,extreme temperature _amt,landslide_amt,volcanic activity_amt,drought_amt,mass movement (dry)_amt,...,era_12_minimum_2m_air_temperature_y,era_12_maximum_2m_air_temperature_y,era_12_dewpoint_2m_temperature_y,era_12_total_precipitation_y,era_12_surface_pressure_y,era_12_mean_sea_level_pressure_y,era_12_u_component_of_wind_10m_y,era_12_v_component_of_wind_10m_y,era_lat_y,era_long_y
0,"(-46, 168)",1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(-46, 168)",1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(-46, 168)",1962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(-46, 168)",1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(-46, 168)",1964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45873,"(66, 130)",2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45874,"(66, 130)",2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45875,"(66, 130)",2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45876,"(66, 130)",2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Run Experiments 

In [126]:
# pwd
import sys
# setting path
sys.path.append('../src')

#import model training module 
import models as m 

#split training and testing 
from sklearn.model_selection import train_test_split
# import shap
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

In [85]:
#attach target for a particular disease for next n years, using y_master  
#next_n is how we choose the next n-periods for the prediction target
def attach_target(x_df, y_master, disaster, next_n): 
    y = y_master.copy()
    #shift years
    y['year'] = y['year'] - next_n
    #keep for particular disaster 
    y = y[['grid_id','year',disaster+'_bin']]
    # Rename into target
    y = y.rename(columns={disaster +'_bin': 'target_' + disaster + '_'+ str(next_n)})
    xy_df = pd.merge(x_df, y, on = ['grid_id','year'], how='inner')
    return xy_df


In [143]:
#construct x_df with various modalities: stat, nlp, era features 

#select to those id pass the filtering criteria 
x_df = df_yrly.loc[df_yrly['grid_id'].isin(grid_id_ls)]
#interpolate missing years to have no flood 
x_df = pd.merge(idd, x_df, on=['grid_id','year'], how='left').fillna(0)

print('length of x_df', len(x_df))

#attach nlp features 
x_df = attach_nlp(x_df, df_nlp)
# #attach era features 
# x_df = attach_era(x_df, df_era)


length of x_df 46669
shape of xy_df with nlp features (46669, 58)


In [146]:
x_df
#attach era features 
x_df = attach_era(x_df, df_era)
x_df

shape of xy_df with era features (46669, 278)


,grid_id,year,flood_amt,storm_amt,earthquake_amt,extreme temperature _amt,landslide_amt,volcanic activity_amt,drought_amt,mass movement (dry)_amt,...,era_12_minimum_2m_air_temperature_y,era_12_maximum_2m_air_temperature_y,era_12_dewpoint_2m_temperature_y,era_12_total_precipitation_y,era_12_surface_pressure_y,era_12_mean_sea_level_pressure_y,era_12_u_component_of_wind_10m_y,era_12_v_component_of_wind_10m_y,era_lat_y,era_long_y
0,"(-46, 168)",1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(-46, 168)",1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(-46, 168)",1962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(-46, 168)",1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(-46, 168)",1964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46664,"(66, 130)",2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46665,"(66, 130)",2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46666,"(66, 130)",2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46667,"(66, 130)",2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
df_era

,1_mean_2m_air_temperature,1_minimum_2m_air_temperature,1_maximum_2m_air_temperature,1_dewpoint_2m_temperature,1_total_precipitation,1_surface_pressure,1_mean_sea_level_pressure,1_u_component_of_wind_10m,1_v_component_of_wind_10m,2_mean_2m_air_temperature,...,12_dewpoint_2m_temperature,12_total_precipitation,12_surface_pressure,12_mean_sea_level_pressure,12_u_component_of_wind_10m,12_v_component_of_wind_10m,year,lat,long,grid_id
0,280.732391,276.481750,286.436157,277.461914,0.182955,97196.718750,100086.515625,4.649180,-0.568458,282.632080,...,276.148590,0.133512,97371.992188,100277.859375,4.096223,-0.112465,1990,-54,-72,"(-54, -72)"
1,279.195557,271.100372,290.727417,276.204803,0.287271,90187.007812,100793.093750,2.730071,-1.450440,281.566498,...,275.571167,0.253060,90203.921875,100820.429688,2.443160,-1.339134,2017,-49,-73,"(-49, -73)"
2,286.905273,275.971741,299.562683,274.170776,0.021335,92155.750000,100647.632812,5.395926,-1.747644,286.351410,...,269.993988,0.032401,91627.937500,100194.570312,5.218695,-1.139208,1991,-49,-70,"(-49, -70)"
3,283.893646,274.633179,295.927734,279.956207,0.112344,93330.679688,101365.687500,1.874738,-0.753395,282.869995,...,278.860962,0.193761,93280.343750,101359.429688,2.096800,-0.883072,2007,-46,-73,"(-46, -73)"
4,282.626709,274.491669,291.303619,279.329773,0.305667,93043.046875,101109.804688,2.263980,-1.185996,282.552338,...,277.970123,0.245718,92992.773438,101092.289062,2.298511,-0.999548,2010,-46,-73,"(-46, -73)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7972,235.834885,222.739182,251.434311,232.512177,0.006969,93440.148438,102651.304688,0.514051,0.670460,240.236725,...,229.950821,0.004996,92814.562500,101953.828125,0.499564,1.215622,1998,66,130,"(66, 130)"
7973,238.504425,223.017487,259.830963,235.215561,0.003875,92749.226562,101773.632812,1.406846,1.488987,235.905975,...,234.151413,0.004338,92871.890625,101969.570312,0.655941,1.295936,2007,66,130,"(66, 130)"
7974,236.635864,224.434296,254.942383,233.412125,0.005831,93840.953125,103060.304688,0.698611,1.081600,238.844528,...,234.401718,0.005005,93443.507812,102624.578125,1.094045,0.911866,2012,66,130,"(66, 130)"
7975,245.251770,231.130249,258.097565,241.889053,0.007374,101070.218750,101287.226562,-0.455457,0.971647,254.961594,...,251.950745,0.013709,100066.625000,100276.078125,-0.825087,0.420770,2006,68,-135,"(68, -135)"


In [130]:
#construct xy_df, depending on the n_pred target period 
n_pred = 1

xy_df = attach_target(x_df, y_master, 'flood', n_pred)
print('length of xy_df', len(xy_df))
print('imbalance', xy_df.filter(regex='target').sum()/len(xy_df))

#construct x,y train and test set 
x = xy_df.drop(xy_df.filter(regex='target').columns, axis=1)#drop target col 
x = x.select_dtypes(['number'])#drop index col
y = xy_df.filter(regex='target') #filter to cols containing target 

length of xy_df 45878
imbalance target_flood_1    0.060639
dtype: float64


In [135]:
#stat features only 
x_stat = x.drop(x.filter(regex='nlp').columns, axis=1)#drop nlp features  
x_stat = x_stat.drop(x.filter(regex='era').columns, axis=1)#drop era features 

#stat + era 
x_stat_era = x.drop(x.filter(regex='nlp').columns, axis=1)#drop nlp features  

#stat + nlp 
x_stat_nlp = x.drop(x.filter(regex='era').columns, axis=1)#drop nlp features  

#stat + nlp + era 
x_stat_nlp_era = x.copy()

In [133]:
def run_all_models(x, y):
    results={}

    #train_test_split 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    print(x_train.shape) #x_train contains stat features only 
    
    #try a dumb model predicting just zeros , we are just guessing no flood!
    y_base = np.zeros(len(y_test)) 
    print('Dumb baseline AUC', metrics.roc_auc_score(y_test,y_base))
    results['base1']=m.get_scores_clf(y_test,y_base)

    #try a less dumb model predicting outcome = current flood outcome 
    y_base2 = x_test['flood_bin'] 
    print('Dumb baseline 2 AUC', metrics.roc_auc_score(y_test,y_base2))
    results['base2']= m.get_scores_clf(y_test,y_base2)
    
    #logreg model 
    y_pred, y_pred_prob = m.run_logreg(x_train, y_train, x_test)
    results['logreg'] = m.get_scores_clf(y_test, y_pred_prob)
    # print('Test AUC', metrics.roc_auc_score(y_test, y_pred_prob))
    
    y_pred, y_pred_prob = m.run_xgb(x_train, y_train, x_test)
    results['xgb'] = m.get_scores_clf(y_test, y_pred_prob)

    return results 


In [136]:
for x_i in [x_stat_nlp_era, x_stat, x_stat_era, x_stat_nlp]: 
    print('x_i shape', x_i.shape)
    run_all_models(x_i,y)

(45878, 167)
(32114, 167)
Dumb baseline AUC 0.5
maximum f1 score, thres 0.48441714114474077 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5 0.48441714114474077 0.9395524556814879 0.5 0.06044754431851206 0.0
[[12932     0]
 [  832     0]]
Dumb baseline 2 AUC 0.5551864114658926
maximum f1 score, thres 0.5559981154419161 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5551864114658926 0.5559981154419161 0.9006102877070619 0.5551864114658925 0.07781682803007245 0.1622596153846154
[[12261   671]
 [  697   135]]
running log reg...


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [137]:
x


,year,flood_amt,storm_amt,earthquake_amt,extreme temperature _amt,landslide_amt,volcanic activity_amt,drought_amt,mass movement (dry)_amt,flood_ct,...,era_12_minimum_2m_air_temperature,era_12_maximum_2m_air_temperature,era_12_dewpoint_2m_temperature,era_12_total_precipitation,era_12_surface_pressure,era_12_mean_sea_level_pressure,era_12_u_component_of_wind_10m,era_12_v_component_of_wind_10m,era_lat,era_long
0,1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45873,2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45874,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45875,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45876,2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
#try a simple baseline

#try a dumb model predicting just zeros , we are just guessing no flood!
y_base = np.zeros(len(y_test)) 
print('Dumb baseline AUC', metrics.roc_auc_score(y_test,y_base))
results['base1']=m.get_scores_clf(y_test,y_base)

#try a less dumb model predicting outcome = current flood outcome 
y_base2 = x_test['flood_bin'] 
print('Dumb baseline 2 AUC', metrics.roc_auc_score(y_test,y_base2))
results['base2']= m.get_scores_clf(y_test,y_base2)



Dumb baseline AUC 0.5
maximum f1 score, thres 0.48441714114474077 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5 0.48441714114474077 0.9395524556814879 0.5 0.06044754431851206 0.0
[[12932     0]
 [  832     0]]
Dumb baseline 2 AUC 0.5551864114658926
maximum f1 score, thres 0.5559981154419161 0.4
auc, f1, accu, accu_bl, precision, recall=  0.5551864114658926 0.5559981154419161 0.9006102877070619 0.5551864114658925 0.07781682803007245 0.1622596153846154
[[12261   671]
 [  697   135]]


In [134]:
#logreg model -> How is this doing the same thing as our baseline model
y_pred, y_pred_prob = m.run_logreg(x_train, y_train, x_test)
results['logreg'] = m.get_scores_clf(y_test, y_pred_prob)
# print('Test AUC', metrics.roc_auc_score(y_test, y_pred_prob))


running log reg...


/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:432: RuntimeWarning: divide by zero encountered in double_scalars
  l2_reg_strength = 1.0 / C
/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/linear_model/_linear_loss.py:198: RuntimeWarning: invalid value encountered in double_scalars
  loss += 0.5 * l2_reg_stre

Train AUC:  0.5
maximum f1 score, thres 0.49608075131257856 0.6
auc, f1, accu, accu_bl, precision, recall=  0.5 0.49608075131257856 0.984444933597476 0.5 0.015555066402524029 0.0
[[13417     0]
 [  212     0]]


/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings

In [125]:
y_pred, y_pred_prob = m.run_xgb(x_train, y_train, x_test)
results['xgb'] = m.get_scores_clf(y_test, y_pred_prob)


running xgb...


/home/gridsan/czeng12/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/gridsan/czeng12/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/gridsan/czeng12/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/gridsan/czeng12/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

Train AUC:  0.7947773028810027
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5641563808212884 0.6
auc, f1, accu, accu_bl, precision, recall=  0.7817612262515168 0.5277838608784438 0.8338419064225516 0.6264909720074234 0.09714549547823817 0.390625
[[11152  1780]
 [  507   325]]


In [206]:
print('Test AUC', metrics.roc_auc_score(y_test, y_pred_prob))

Test AUC 0.9651197579528084


In [207]:
results= pd.DataFrame(results)
# results.to_clipboard(excel=True)
results

,base1,base2,logreg,xgb
0,0.500000,0.786129,0.500000,0.965120
1,0.496081,0.615005,0.496081,0.639297
2,0.984445,0.944530,0.984445,0.975347
3,0.500000,0.786129,0.500000,0.655543
4,0.015555,0.107588,0.015555,0.096208
5,0.000000,0.622642,0.000000,0.325472


## Compare results from different combinations of x_train, x_train1, etc.  
- correspond to different modalities of data:
   - try which ones give the best result 
   - @Oscar you can take over from here :) 

In [ ]:
## to do: 
#- duplicate this notebook keep only to ee feature extraction 
#- in a new notebook to multimodal testing 